In [8]:
from math import sqrt
import copy
import  traceback
import shutil
import random

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm
import openpyxl

In [2]:
def load_scan(path):
    slices = [] #slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    for s in os.listdir(path):
        if os.path.isdir(os.path.join(path, s)): #if not s.endswith('.dcm'):
            continue
        sl = pydicom.dcmread(os.path.join(path, s))
        try:
            sl_p = sl.pixel_array
        except AttributeError:
            traceback.print_exc()
            print(f'\tDelete {os.path.join(path, s)}')
            os.remove(os.path.join(path, s))
        else:
            slices.append(sl)
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

In [3]:
window_width, window_level = 600, 200
lower_b, upper_b = window_level - window_width//2, window_level + window_width//2
print(lower_b, upper_b)

-100 500


## 1.阴性数据

In [4]:
# 打印哪个病例没有2
def print_no_cta(input_dir):
    for patient in os.listdir(input_dir):
        patient_path = os.path.join(input_dir, patient)
        if os.path.isfile(patient_path): continue
        if '2' not in os.listdir(patient_path): print(patient_path)
            
print_no_cta('/nfs3-p2/zsxm/dataset/2021-9-17-negative')

/nfs3-p2/zsxm/dataset/2021-9-17-negative/wanghaifang-40-47-83-160


In [10]:
# 将2下的dcm文件根据窗宽窗位转化为png图片
def generate_image(input_folder):
    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '2':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))
            print_flag = False
            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                if img.height != img.width:
                    if not print_flag:
                        print(patient, 'height not equal to width\n')
                        print_flag = True
                    height = width = min(img.height, img.width)
                    if img.height != height:
                        start = (img.height - height) / 2
                        img = img.crop((0, start, img.width, start + height))
                    elif img.width != width:
                        start = (img.width - width) / 2
                        img = img.crop((start, 0, start + height, img.height))
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

generate_image('/nfs3-p1/zsxm/dataset/2021-9-17-negative/')

****Processing baixiaoxu-22-31-52-147****
****Processing baojinjia-17-21-35-83****
****Processing baoyanzu-21-25-42-93****
****Processing bianyinqiao-31-38-59-137****
****Processing bianzhaorong-17-22-38-92****
****Processing caijingnan1-41-50-73-184****
****Processing caiqishu-34-43-70-176****
****Processing caiyouzhe-14-18-32-83****
****Processing caoyuxia-16-21-35-87****
****Processing chenfugui-36-46-76-149****
****Processing chengzhiqiang-33-41-67-159****
****Processing chenjian-26-30-44-89****
****Processing chenjianhua-14-17-32-80****
****Processing chenjianjun-25-34-65-175****
****Processing chenjianping-15-22-41-91****
****Processing chenjingjing-38-47-67-184****
****Processing chenjinmei-12-17-34-80****
****Processing chenpanyang-21-31-71-161****
****Processing chenqiuying-13-17-30-78****
****Processing chensaimu-34-44-78-166****
****Processing chenxiufen-17-21-34-82****
****Processing chenyanli-20-23-34-80****
****Processing chenyinfen-18-22-38-81****
****Processing chesheng

In [11]:
# 将各个病例中的png图片文件夹统一移动到一起供yolov5检测
def move_together_for_detect(input_folder, dst_path):
    workbook_path = os.path.join(input_folder, 'label.xlsx')
    wb = openpyxl.load_workbook(workbook_path)
    sheet = wb['Sheet1']
    
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in sorted(os.listdir(input_folder)):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        flag = True
        for row in sheet.iter_rows():
            if row[0].value == patient.split('-')[0]:
                if row[3].value is not None and row[4].value is not None:
                    flag = False
                    
                    
                break
        if flag: continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '2', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/9_detect/')

****Processing baixiaoxu-22-31-52-147****
****Processing baojinjia-17-21-35-83****
****Processing baoyanzu-21-25-42-93****
****Processing bianyinqiao-31-38-59-137****
****Processing bianzhaorong-17-22-38-92****
****Processing caijingnan1-41-50-73-184****
****Processing caiyouzhe-14-18-32-83****
****Processing caoyuxia-16-21-35-87****
****Processing chenfugui-36-46-76-149****
****Processing chengzhiqiang-33-41-67-159****
****Processing chenjian-26-30-44-89****
****Processing chenjianhua-14-17-32-80****
****Processing chenjianjun-25-34-65-175****
****Processing chenjianping-15-22-41-91****
****Processing chenjingjing-38-47-67-184****
****Processing chenjinmei-12-17-34-80****
****Processing chenpanyang-21-31-71-161****
****Processing chenqiuying-13-17-30-78****
****Processing chensaimu-34-44-78-166****
****Processing chenxiufen-17-21-34-82****
****Processing chenyanli-20-23-34-80****
****Processing chenyinfen-18-22-38-81****
****Processing cheshengying-25-34-64-139****
****Processing deng